# Deployment with BERT



Pre-trained language representations have been shown to improve many downstream NLP tasks such as question answering, and natural language inference. Devlin, Jacob, et al proposed BERT [1] (Bidirectional Encoder Representations from Transformers), which fine-tunes deep bidirectional representations on a wide range of tasks with minimal task-specific parameters, and obtained state- of-the-art results.

In this tutorial, we will focus on adapting the BERT model for the question answering task on the SQuAD dataset. Specifically, we will:

- understand how to pre-process the SQuAD dataset to leverage the learnt representation in BERT,
- adapt the BERT model to the question answering task, and
- load a trained model to perform inference on the SQuAD dataset


## Load MXNet and GluonNLP

We first import the libraries:

In [13]:
import collections, time, logging
import numpy as np
import gluonnlp as nlp
import mxnet as mx
import bert
import qa_utils
from bert.bert_qa_evaluate import PredResult, predict
import tvm
from tvm import relay
import topi

## Inspect the SQuAD Dataset

Then we take a look at the Stanford Question Answering Dataset (SQuAD). The dataset can be downloaded using the `nlp.data.SQuAD` API. In this tutorial, we create a small dataset with 3 samples from the SQuAD dataset for demonstration purpose.

The question answering task on the SQuAD dataset is setup the following way. For each sample in the dataset, a context is provided. The context is usually a long paragraph which contains lots of information. Then a question asked based on the context. The goal is to find the text span in the context that answers the question in the sample.

In [2]:
full_data = nlp.data.SQuAD(segment='dev', version='1.1')
# loading a subset of the dev set of SQuAD
num_target_samples = 3
target_samples = [full_data[i] for i in range(num_target_samples)]
dataset = mx.gluon.data.SimpleDataset(target_samples)
print('Number of samples in the created dataset subsampled from SQuAD = %d'%len(dataset))

Number of samples in the created dataset subsampled from SQuAD = 3


Let's take a look at a sample from the dataset. In this sample, the question is about the location of the game, with a description about the Super Bowl 50 game as the context. Note that three different answer spans are correct for this question, and they start from index 403, 355 and 355 in the context respectively.

In [3]:
sample = dataset[2]

context_idx = 3

print('\nContext:')
print(sample[context_idx])


Context:
Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.


In [4]:
question_idx = 2
answer_idx = 4
answer_pos_idx = 5

print("\nQuestion")
print(sample[question_idx])
print("\nCorrect Answer Spans")
print(sample[answer_idx])
print("\nAnswer Span Start Indices:")
print(sample[answer_pos_idx])


Question
Where did Super Bowl 50 take place?

Correct Answer Spans
['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."]

Answer Span Start Indices:
[403, 355, 355]


## Data Pre-processing for QA with BERT

Recall that during BERT pre-training, it takes a sentence pair as the input, separated by the 'SEP' special token. For SQuAD, we can feed the context-question pair as the sentence pair input. To use BERT to predict the starting and ending span of the answer, we can add a classification layer for each token in the context texts, to predict if a token is the start or the end of the answer span. 

In the next few code blocks, we will work on pre-processing the samples in the SQuAD dataset in the desired format with these special separators. 


### Tokenization

First, let's use the *get_model* API in GluonNLP to get the model definition for BERT, and the vocabulary used for the BERT model. Note that we discard the pooler and classifier layers used for the next sentence prediction task, as well as the decoder layers for the masked language model task during the BERT pre-training phase. These layers are not useful for predicting the starting and ending indices of the answer span.

In [16]:
from bert.export.hybrid_bert import HybridBERTForQA, get_hybrid_model
seq_length = 256
bert, vocab = get_hybrid_model(
    name="bert_12_768_12",
    dataset_name="book_corpus_wiki_en_uncased",
    pretrained=False,
    use_pooler=False,
    use_decoder=False,
    use_classifier=False,
    seq_length=seq_length)

Note that there are several special tokens in the vocabulary for BERT. In particular, the `[SEP]` token is used for separating the sentence pairs, and the `[CLS]` token is added at the beginning of the sentence pairs. They will be used to pre-process the SQuAD dataset later.

In [17]:
print(vocab)

Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")


The second step is to process the samples using the same tokenizer used for BERT, which is provided as the `BERTTokenizer` API in GluonNLP. Note that instead of word level and character level representation, BERT uses subwords to represent a word, separated `##`. 

In the following example, the word `suspending` is tokenized as two subwords (`suspend` and `##ing`), and `numerals` is tokenized as three subwords (`nu`, `##meral`, `##s`).

In [18]:
tokenizer = nlp.data.BERTTokenizer(vocab=vocab, lower=True)

tokenizer("as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals")

['as',
 'well',
 'as',
 'temporarily',
 'suspend',
 '##ing',
 'the',
 'tradition',
 'of',
 'naming',
 'each',
 'super',
 'bowl',
 'game',
 'with',
 'roman',
 'nu',
 '##meral',
 '##s']

### Sentence Pair Composition

With the tokenizer inplace, we are ready to process the question-context texts and compose sentence pairs. The functionality is available via the `SQuADTransform` API. 

In [8]:
transform = bert.data.qa.SQuADTransform(tokenizer, is_pad=False, is_training=False, do_lookup=False)
dev_data_transform, _ = bert.data.qa.preprocess_dataset(dataset, transform)
logging.info('The number of examples after preprocessing:{}'.format(len(dev_data_transform)))

Done! Transform dataset costs 0.10 seconds.


Let's take a look at the sample after the transformation:

In [9]:
sample = dev_data_transform[2]
print('\nsegment type: ' + str(sample[2]))
print('\ntext length: ' + str(sample[3]))
print('\nsentence pair: ' + str(sample[1]))


segment type: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

text length: 168

sentence pair: ['[CLS]', 'where', 'did', 'super', 'bowl', '50', 'take', 'place', '?', '[SEP]', 'super', 'bowl', '50', 'was', 'an', 'american', 'football', 'game', 'to', 'determine', 'the', 'champion', 'of', 'the', 'national', 'football', 'league', '(', 'nfl', ')', 'for', 'the', '2015', 'season', '.', 'the', 'american', 'football', 'conference', '(', 'afc', ')', 'champion', 'denver', 'broncos', 'defeated', 'the', 'national', 'football', 'conference', '(', 'n

### Vocabulary Lookup

Finally, we convert the transformed texts to subword indices, which are used to contructor NDArrays as the inputs to the model.

In [10]:
def vocab_lookup(example_id, subwords, type_ids, length, start, end):
    indices = vocab[subwords]
    return example_id, indices, type_ids, length, start, end

dev_data_transform = dev_data_transform.transform(vocab_lookup, lazy=False)
print(dev_data_transform[2][1])

[2, 2073, 2106, 3565, 4605, 2753, 2202, 2173, 1029, 3, 3565, 4605, 2753, 2001, 2019, 2137, 2374, 2208, 2000, 5646, 1996, 3410, 1997, 1996, 2120, 2374, 2223, 1006, 5088, 1007, 2005, 1996, 2325, 2161, 1012, 1996, 2137, 2374, 3034, 1006, 10511, 1007, 3410, 7573, 14169, 3249, 1996, 2120, 2374, 3034, 1006, 22309, 1007, 3410, 3792, 12915, 2484, 1516, 2184, 2000, 7796, 2037, 2353, 3565, 4605, 2516, 1012, 1996, 2208, 2001, 2209, 2006, 2337, 1021, 1010, 2355, 1010, 2012, 11902, 1005, 1055, 3346, 1999, 1996, 2624, 3799, 3016, 2181, 2012, 4203, 10254, 1010, 2662, 1012, 2004, 2023, 2001, 1996, 12951, 3565, 4605, 1010, 1996, 2223, 13155, 1996, 1000, 3585, 5315, 1000, 2007, 2536, 2751, 1011, 11773, 11107, 1010, 2004, 2092, 2004, 8184, 28324, 2075, 1996, 4535, 1997, 10324, 2169, 3565, 4605, 2208, 2007, 3142, 16371, 28990, 2015, 1006, 2104, 2029, 1996, 2208, 2052, 2031, 2042, 2124, 2004, 1000, 3565, 4605, 1048, 1000, 1007, 1010, 2061, 2008, 1996, 8154, 2071, 14500, 3444, 1996, 5640, 16371, 28990, 2015

We download a BERT model trained on the SQuAD dataset, prepare the dataloader.

In [19]:
net = HybridBERTForQA(bert)
ctx = mx.cpu()
ckpt = qa_utils.download_qa_ckpt()
net.load_parameters(ckpt, ctx=ctx)

# batch_size = 1
# dev_dataloader = mx.gluon.data.DataLoader(
#     dev_data_transform, batch_size=batch_size, shuffle=False)

Downloaded checkpoint to ./temp/bert_qa-7eb11865.params


In [20]:
shape_dict = {
    'data0': (1, seq_length),
    'data1': (1, seq_length),
    'data2': (1,)
}
mod, params = relay.frontend.from_mxnet(net, shape_dict)
print(mod)

v0.0.1
def @main(%data2: Tensor[(1,), float32], %hybridbertmodel2_word_embed_embedding0_weight: Tensor[(30522, 768), float32], %data0: Tensor[(1, 256), float32], %hybridbertmodel2_token_type_embed_embedding0_weight: Tensor[(2, 768), float32], %data1: Tensor[(1, 256), float32], %hybridbertencoder2_position_weight: Tensor[(512, 768), float32], %hybridbertencoder2_bertlayernorm0_gamma: Tensor[(768,), float32], %hybridbertencoder2_bertlayernorm0_beta: Tensor[(768,), float32], %hybridbertencoder2_transformer0_multiheadattentioncell0_query_weight: Tensor[(768, 768), float32], %hybridbertencoder2_transformer0_multiheadattentioncell0_query_bias: Tensor[(768,), float32], %hybridbertencoder2_transformer0_multiheadattentioncell0_key_weight: Tensor[(768, 768), float32], %hybridbertencoder2_transformer0_multiheadattentioncell0_key_bias: Tensor[(768,), float32], %hybridbertencoder2_transformer0_multiheadattentioncell0_value_weight: Tensor[(768, 768), float32], %hybridbertencoder2_transformer0_multih

In [13]:
all_results = collections.defaultdict(list)

total_num = 0
for data in dev_dataloader:
    example_ids, inputs, token_types, valid_length, _, _ = data
    total_num += len(inputs)
    batch_size = inputs.shape[0]
    pred_start, pred_end = net(inputs.astype('float32').as_in_context(ctx),
                               token_types.astype('float32').as_in_context(ctx),
                               valid_length.astype('float32').as_in_context(ctx))

    example_ids = example_ids.asnumpy().tolist()
    pred_start = pred_start.reshape(batch_size, -1).asnumpy()
    pred_end = pred_end.reshape(batch_size, -1).asnumpy()
    
    for example_id, start, end in zip(example_ids, pred_start, pred_end):
        all_results[example_id].append(PredResult(start=start, end=end))

In [14]:
qa_utils.predict(dataset, all_results, vocab)


Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.

Question: which nfl team represented the afc at super bowl 50 ?

Top predictions: 
99.36% 	 Denver Broncos
0.23% 	 The American Football Conference (AFC) champion Denver Broncos
0.20% 	 Broncos


Context: Super Bo